##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Create a TFX pipeline using templates with Local orchestrator

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/template_local"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で実行</a></td>
<td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/tfx/template_local.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colabで実行</a>
</td>
<td><a target="_blank" href="https://github.com/tensorflow/tfx/tree/master/docs/tutorials/tfx/template_local.ipynb">     <img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     GitHubでソースを表示</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/tfx/docs/tutorials/tfx/template_local.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table></div>

## 重み値のみを保存。（通常、モデルのトレーニング時に使用）。

This document will provide instructions to create a TensorFlow Extended (TFX) pipeline using *templates* which are provided with TFX Python package. Most of instructions are Linux shell commands, and corresponding Jupyter Notebook code cells which invoke those commands using `!` are provided.

You will build a pipeline using [Taxi Trips dataset](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew) released by the City of Chicago. We strongly encourage you to try to build your own pipeline using your dataset by utilizing this pipeline as a baseline.

We will build a pipeline which runs on local environment. If you are interested in using Kubeflow orchestrator on Google Cloud, please see [TFX on Cloud AI Platform Pipelines tutorial](https://www.tensorflow.org/tfx/tutorials/tfx/cloud-ai-platform-pipelines).

## 前提条件

- Linux / MacOS
- Python &gt;= 3.5.3

You can get all prerequisites easily by [running this notebook on Google Colab](https://colab.sandbox.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/tfx/template_local.ipynb).


## ステップ 1.  環境をセットアップする

**Throughout this document, we will present commands twice. Once as a copy-and-paste-ready shell command, once as a jupyter notebook cell. If you are using Colab, just skip shell script block and execute notebook cells.**

You should prepare a development environment to build a pipeline.

Install `tfx` python package. We recommend use of `virtualenv` in the local environment. You can use following shell script snippet to set up your environment.

```sh
# Create a virtualenv for tfx.
virtualenv -p python3 venv
source venv/bin/activate
# Install python packages.
python -m pip install --upgrade "tfx<2"
```

If you are using colab:


In [ ]:
import sys
!{sys.executable} -m pip install --upgrade "tfx<2"

NOTE: There might be some errors during package installation. For example,

>ERROR: some-package 0.some_version.1 has requirement other-package!=2.0.,&lt;3,&gt;=1.15, but you'll have other-package 2.0.0 which is incompatible.

Please ignore these errors at this moment.

In [ ]:
# Set `PATH` to include user python binary directory.
HOME=%env HOME
PATH=%env PATH
%env PATH={PATH}:{HOME}/.local/bin

Let's check the version of TFX.
```bash
python -c "from tfx import version ; print('TFX version: {}'.format(version.__version__))"
```

In [ ]:
!python3 -c "from tfx import version ; print('TFX version: {}'.format(version.__version__))"

以上でパイプラインを作成する準備ができました。

## Step 2. Copy predefined template to your project directory.

このステップでは、事前定義されたテンプレートから追加のファイルをコピーして、作業パイプラインプロジェクトディレクトリとファイルを作成します。

以下の `PIPELINE_NAME` を変更することで、パイプラインに別の名前を付けることができます。また、これはファイルが配置されるプロジェクトディレクトリ名にもなります。

```bash
export PIPELINE_NAME="my_pipeline"
export PROJECT_DIR=~/tfx/${PIPELINE_NAME}
```

In [ ]:
PIPELINE_NAME="my_pipeline"
import os
# Create a project directory under Colab content directory.
PROJECT_DIR=os.path.join(os.sep,"content",PIPELINE_NAME)

TFX には、TFX python パッケージと `taxi` テンプレートが含まれています。分類や回帰など、ポイントワイズの予測問題を解決するには、このテンプレートを利用してスタートできます。

`tfx template copy` CLI コマンドは、事前定義されたテンプレートファイルをプロジェクトディレクトリにコピーします。

```sh
tfx template copy \
   --pipeline_name="${PIPELINE_NAME}" \
   --destination_path="${PROJECT_DIR}" \
   --model=taxi
```

In [ ]:
!tfx template copy \
  --pipeline_name={PIPELINE_NAME} \
  --destination_path={PROJECT_DIR} \
  --model=taxi

このノートブックの作業ディレクトリコンテキストをプロジェクトディレクトリに変更します。

```bash
cd ${PROJECT_DIR}
```

In [ ]:
%cd {PROJECT_DIR}

## Step 3. Browse your copied source files.

TFX テンプレートは、Python ソースコード、サンプルデータ、パイプラインの出力を分析する Jupyter Notebook など、パイプラインを構築するための基本的なスキャフォールドファイルを提供します。`taxi` テンプレートは、<a>Airflow Tutorial </a>と同じ<em>シカゴタクシー</em>データセットと ML モデルを使用します。

In Google Colab, you can browse files by clicking a folder icon on the left. Files should be copied under the project directoy, whose name is `my_pipeline` in this case. You can click directory names to see the content of the directory, and double-click file names to open them.

ここでは、各 Python ファイルを簡単に紹介します。

- pipeline - このディレクトリには、パイプラインの定義が含まれています。
    - configs.py — パイプライン ランナーの共通定数を定義します。
    - pipeline.py — TFX コンポーネントとパイプラインを定義します。
- models - このディレクトリには、機械学習モデルの定義が含まれています。
    - `features.py`、`features_test.py` — モデルの特徴を定義します。
    - `preprocessing.py`、`preprocessing_test.py` — `tf::Transform`を使用して前処理ジョブを定義します。
    - estimator - このディレクトリには、Estimator ベースのモデルが含まれています。
        - constants.py — モデルの定数を定義します。
        - `model.py`、`model_test.py` — TF estimator を使用して DNN モデルを定義します
    - keras - このディレクトリには、Keras ベースのモデルが含まれています。
        - constants.py — モデルの定数を定義します。
        - `model.py`、`model_test.py` — Keras を使用して DNN モデルを定義します。
- `beam_runner.py`、`kubeflow_runner.py` — オーケストレーション エンジンごとにランナーを定義します。


名前に `_test.py` が含まれているファイルがあることに気が付きましたか。これらはパイプラインの単体テストであり、独自のパイプラインを実装するときに単体テストを追加することをお勧めします。テストファイルのモジュール名に `-m` フラグを指定すると、単体テストを実行できます。通常、モジュール名を取得するには、`.py` 拡張子を削除し、`/` を `.` に置き換えます。以下に例を示します。

```bash
python -m models.features_test
```

In [ ]:
!{sys.executable} -m models.features_test
!{sys.executable} -m models.keras.model_test


## ステップ 4. 最初の TFX パイプラインを実行する

You can create a pipeline using `pipeline create` command.

```bash
tfx pipeline create --engine=local --pipeline_path=local_runner.py
```


In [ ]:
!tfx pipeline create --engine=local --pipeline_path=local_runner.py

Then, you can run the created pipeline using `run create` command.
```sh
tfx run create --engine=local --pipeline_name="${PIPELINE_NAME}"
```

In [ ]:
!tfx run create --engine=local --pipeline_name={PIPELINE_NAME}

If successful, you'll see `Component CsvExampleGen is finished.` When you copy the template, only one component, CsvExampleGen, is included in the pipeline.

## ステップ 5. データ検証用のコンポーネントを追加する

このステップでは、`StatisticsGen`、`SchemaGen`、および、`ExampleValidator` などのデータ検証用のコンポーネントを追加します。データ検証については、[Tensorflow データ検証の基礎](https://www.tensorflow.org/tfx/data_validation/get_started)を参照してください。

We will modify copied pipeline definition in `pipeline/pipeline.py`. If you are working on your local environment, use your favorite editor to edit the file. If you are working on Google Colab,

> **Click folder icon on the left to open `Files` view**.

> **Click `my_pipeline` to open the directory and click `pipeline` directory to open and double-click `pipeline.py` to open the file**.

> Find and uncomment the 3 lines which add `StatisticsGen`, `SchemaGen`, and `ExampleValidator` to the pipeline. (Tip: find comments containing `TODO(step 5):`).

> Your change will be saved automatically in a few seconds. Make sure that the `*` mark in front of the `pipeline.py` disappeared in the tab title. **There is no save button or shortcut for the file editor in Colab. Python files in file editor can be saved to the runtime environment even in `playground` mode.**

次に、パイプライン定義を変更して既存のパイプラインを更新します。`tfx pipeline update` コマンドを使用してパイプラインを更新し、続いて `tfx run create` コマンドを使用して更新されたパイプラインの新しい実行を作成します。

```sh
# Update the pipeline
tfx pipeline update --engine=local --pipeline_path=local_runner.py
# You can run the pipeline the same way.
tfx run create --engine local --pipeline_name "${PIPELINE_NAME}"
```


In [ ]:
# Update the pipeline
!tfx pipeline update --engine=local --pipeline_path=local_runner.py
# You can run the pipeline the same way.
!tfx run create --engine local --pipeline_name {PIPELINE_NAME}

You should be able to see the output log from the added components. Our pipeline creates output artifacts in `tfx_pipeline_output/my_pipeline` directory.

## ステップ 6. トレーニング用のコンポーネントを追加する

このステップでは、`Transform`、`Trainer`、`Resolver`、`Evaluator`、および、`Pusher` などのトレーニングとモデル検証用のコンポーネントを追加します。

> **Open `pipeline/pipeline.py`**. Find and uncomment 5 lines which add `Transform`, `Trainer`, `Resolver`, `Evaluator` and `Pusher` to the pipeline. (Tip: find `TODO(step 6):`)

以前と同様に、変更されたパイプライン定義で既存のパイプラインを更新する必要があります。手順はステップ 5 と同じです。`tfx pipeline update` を使用してパイプラインを更新し、`tfx run create` を使用して実行を作成します。

```sh
tfx pipeline update --engine=local --pipeline_path=local_runner.py
tfx run create --engine local --pipeline_name "${PIPELINE_NAME}"
```

In [ ]:
!tfx pipeline update --engine=local --pipeline_path=local_runner.py
!tfx run create --engine local --pipeline_name {PIPELINE_NAME}

When this execution run finishes successfully, you have now created and run your first TFX pipeline using Local orchestrator!

**注意：**パイプライン実行を作成するたびに、入力とパラメーターが変更されていなくても、すべてのコンポーネントが何度も実行されることに気付いたかもしれません。これは時間とリソースの無駄なので、パイプラインキャッシングを使用してこれらの実行をスキップできます。`pipeline.py` の `Pipeline` オブジェクトに `enable_cache=True` を指定することで、キャッシュを有効にできます。


## Step 7. (*Optional*) Try BigQueryExampleGen.

BigQuery は、サーバーレスでスケーラビリティと費用対効果の高いクラウド データ ウェアハウスです。BigQuery は、TFX のトレーニング サンプルのソースとして使用できます。このステップでは、パイプラインにBigQueryExampleGenを追加します。

You need a [Google Cloud Platform](https://cloud.google.com/gcp/getting-started) account to use BigQuery. Please prepare a GCP project.

Login to your project using colab auth library or `gcloud` utility.

```sh
# You need `gcloud` tool to login in local shell environment.
gcloud auth login
```

In [ ]:
if 'google.colab' in sys.modules:
  from google.colab import auth
  auth.authenticate_user()
  print('Authenticated')


You should specify your GCP project name to access BigQuery resources using TFX. Set `GOOGLE_CLOUD_PROJECT` environment variable to your project name.

```sh
export GOOGLE_CLOUD_PROJECT=YOUR_PROJECT_NAME_HERE
```

In [ ]:
# Set your project name below.
# WARNING! ENTER your project name before running this cell.
%env GOOGLE_CLOUD_PROJECT=YOUR_PROJECT_NAME_HERE

> **Open `pipeline/pipeline.py`**. Comment out `CsvExampleGen` and uncomment the line which create an instance of `BigQueryExampleGen`. You also need to uncomment `query` argument of the `create_pipeline` function.

We need to specify which GCP project to use for BigQuery again, and this is done by setting `--project` in `beam_pipeline_args` when creating a pipeline.

> **Open `pipeline/configs.py`**. Uncomment the definition of `BIG_QUERY__WITH_DIRECT_RUNNER_BEAM_PIPELINE_ARGS` and `BIG_QUERY_QUERY`. You should replace the project id and the region value in this file with the correct values for your GCP project.

> **Open `local_runner.py`**. Uncomment two arguments, `query` and `beam_pipeline_args`, for create_pipeline() method.

Now the pipeline is ready to use BigQuery as an example source. Update the pipeline and create a run as we did in step 5 and 6.

In [ ]:
!tfx pipeline update --engine=local --pipeline_path=local_runner.py
!tfx run create --engine local --pipeline_name {PIPELINE_NAME}

## What's next: Ingest YOUR data to the pipeline.

シカゴタクシーデータセットを使用してモデルのパイプラインを作成しました。次に、データをパイプラインに入れます。

データは、GCS や BigQuery など、パイプラインがアクセスできる場所であればどこにでも保存できます。データにアクセスするには、パイプライン定義を変更する必要があります。

1. データがファイルに保存されている場合は、`kubeflow_runner.py` または `local_runner.py` の `DATA_PATH` を変更し、ファイルの場所に設定します。データが BigQuery に保存されている場合は、`pipeline/configs.py` の `BIG_QUERY_QUERY` を変更して、データを正しくクエリします。
2. models/features.pyに特徴量を追加します。
3. models/preprocessing.pyを変更して、トレーニング用の入力データを変換します。
4. models/keras/model.pyとmodels/keras/constants.pyを変更して、機械学習モデルを記述します。
    - Estimator ベースのモデルを使用することもできます。 `pipeline/configs.py` の `RUN_FN` 定数を `models.estimator.model.run_fn` に変更します。

詳細については、[トレーナーコンポーネントガイド](https://www.tensorflow.org/tfx/guide/trainer)を参照してください。